In [1]:
!pip install torch einops numpy transformer_lens==2.11.0 


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [39]:
import torch as t
import torch.nn as nn
import einops
import numpy as np
from transformer_lens import (
    ActivationCache,
    FactoredMatrix,
    HookedTransformer,
    HookedTransformerConfig,
    utils,
)
from transformer_lens.hook_points import HookPoint
import tqdm
import csv
import functools
from typing import DefaultDict

device = 'cuda'

In [46]:
gpt2_small = HookedTransformer.from_pretrained('gpt2').to(device)
gpt2_xl = HookedTransformer.from_pretrained('gpt2-XL').to(device)

Loaded pretrained model gpt2 into HookedTransformer
Moving model to device:  cuda


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-XL into HookedTransformer
Moving model to device:  cuda


In [48]:
model = gpt2_xl

### CAA [Sentences]

In [4]:
sentences = []
with open('sentences.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        sentences.append(tuple(row))

In [ ]:
test_cache = model.run_with_cache('')[1]
keys = []
for key in test_cache:
    if ('resid_pre' in key) or ('resid_mid' in key):
        keys.append(key)
print(keys)


['blocks.0.hook_resid_pre', 'blocks.0.hook_resid_mid', 'blocks.1.hook_resid_pre', 'blocks.1.hook_resid_mid', 'blocks.2.hook_resid_pre', 'blocks.2.hook_resid_mid', 'blocks.3.hook_resid_pre', 'blocks.3.hook_resid_mid', 'blocks.4.hook_resid_pre', 'blocks.4.hook_resid_mid', 'blocks.5.hook_resid_pre', 'blocks.5.hook_resid_mid', 'blocks.6.hook_resid_pre', 'blocks.6.hook_resid_mid', 'blocks.7.hook_resid_pre', 'blocks.7.hook_resid_mid', 'blocks.8.hook_resid_pre', 'blocks.8.hook_resid_mid', 'blocks.9.hook_resid_pre', 'blocks.9.hook_resid_mid', 'blocks.10.hook_resid_pre', 'blocks.10.hook_resid_mid', 'blocks.11.hook_resid_pre', 'blocks.11.hook_resid_mid', 'blocks.12.hook_resid_pre', 'blocks.12.hook_resid_mid', 'blocks.13.hook_resid_pre', 'blocks.13.hook_resid_mid', 'blocks.14.hook_resid_pre', 'blocks.14.hook_resid_mid', 'blocks.15.hook_resid_pre', 'blocks.15.hook_resid_mid', 'blocks.16.hook_resid_pre', 'blocks.16.hook_resid_mid', 'blocks.17.hook_resid_pre', 'blocks.17.hook_resid_mid', 'blocks.18.

In [53]:
accum_steering_sentences = t.zeros(len(keys), model.cfg.d_model).to(device)

for (english, spanish) in sentences:
    cache_eng = model.run_with_cache(english)[1]
    cache_spa = model.run_with_cache(spanish)[1]

    for idx, key in enumerate(keys):
        vec_eng = cache_eng[key].squeeze()[-1] #ie. vector at residual-stream-of-last-token
        vec_spa = cache_spa[key].squeeze()[-1]
        steering = vec_spa - vec_eng
        accum_steering_sentences[idx] += steering

In [55]:
steering_vecs_sentences = accum_steering_sentences / 50

def sentences_hook(res_vector, hook, steering):
    return res_vector + steering

for idx, key in enumerate(keys):
    partial_hook = functools.partial(sentences_hook, steering=steering_vecs_sentences[idx])
    logits = model.run_with_hooks(
        'The horse drank some',
        return_type = 'logits',
        fwd_hooks = [(key, partial_hook)],
    )
    out = logits.squeeze().argmax(-1)[-1]
    print(model.to_string(out))


 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 of
 of
 of
 of
 of
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 water
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 of
 por
 of
 un
 un
 of
 of
 un
 un
 un
 un
 por
 por
 un
 un
 un
 un
 por
 por
 un
 un
 por
 por
 por
 por


In [58]:
def test_hook(res_vector, hook, steering):
    return res_vector + 5 * steering

preds = DefaultDict(int)

for (english, spanish) in sentences[:10]:
    cache_eng = gpt2_xl.run_with_cache(english)[1]
    cache_spa = gpt2_xl.run_with_cache(spanish)[1]

    for idx, key in enumerate(keys):
        vec_eng = cache_eng[key].squeeze()[-1]
        vec_spa = cache_spa[key].squeeze()[-1]
        steering = vec_spa - vec_eng

        my_hook = functools.partial(test_hook, steering = steering)
        logits = gpt2_xl.run_with_hooks(
            'The horse drank some',
            return_type = 'logits',
            fwd_hooks = [(key, my_hook)]
        )

        pred = logits.squeeze().argmax(dim=-1)[-1]
        preds[gpt2_xl.to_string(pred)] += 1

# preds should have total of 24 * 50

preds

defaultdict(int,
            {' water': 56,
             'ic': 1,
             ' of': 95,
             'ones': 1,
             ' words': 1,
             '.': 67,
             ',': 79,
             ' SE': 5,
             ' BE': 4,
             ' =': 5,
             ' /': 2,
             ' w': 4,
             ' Sur': 2,
             ' en': 69,
             ' su': 5,
             ' un': 8,
             ' se': 5,
             ' es': 11,
             ' non': 1,
             ' el': 34,
             ' de': 18,
             ' este': 13,
             ' the': 5,
             ' some': 13,
             '\n': 47,
             ' (': 43,
             ' est': 34,
             ' ur': 4,
             ' "': 4,
             'Mi': 3,
             ' entr': 6,
             'senal': 4,
             ' o': 30,
             ' u': 3,
             ' cop': 2,
             ' or': 3,
             ' te': 1,
             ' com': 8,
             ' por': 134,
             ' l': 2,
             ' p': 1,
             ' L':

### CAA [Words]

In [56]:
words = []
with open('words.csv') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        words.append(tuple(row))

In [59]:
def test_hook(res_vector, hook, steering):
    return res_vector + (5 * steering)

preds = DefaultDict(int)

for (english, spanish) in words[:10]:
    cache_eng = model.run_with_cache(english)[1]
    cache_spa = model.run_with_cache(spanish)[1]

    for idx, key in enumerate(keys):
        vec_eng = cache_eng[key].squeeze()[-1]
        vec_spa = cache_spa[key].squeeze()[-1]
        steering = vec_spa - vec_eng

        my_hook = functools.partial(test_hook, steering = steering)
        logits = model.run_with_hooks(
            'The horse drank some',
            return_type = 'logits',
            fwd_hooks = [(key, my_hook)]
        )

        pred = logits.squeeze().argmax(dim=-1)[-1]
        preds[model.to_string(pred)] += 1

# preds should have total of 24 * 50

preds

defaultdict(int,
            {'.': 20,
             'asa': 6,
             ',': 187,
             "'s": 51,
             ' de': 192,
             ' Grande': 18,
             ' Blanc': 2,
             'ro': 5,
             'ze': 4,
             '-': 95,
             '/': 3,
             'pe': 2,
             'ff': 17,
             'ppe': 4,
             '\n': 8,
             'oso': 11,
             '\xad': 6,
             'iac': 6,
             'onga': 6,
             'pload': 2,
             'e': 2,
             ' e': 9,
             ' o': 4,
             ' d': 2,
             ' dé': 2,
             'ux': 2,
             'ú': 1,
             'ister': 1,
             'í': 2,
             ' (': 12,
             ' certainty': 1,
             '�': 27,
             ' con': 3,
             ' del': 39,
             ' por': 2,
             'ña': 12,
             'igo': 1,
             ' U': 5,
             'rer': 23,
             ' to': 27,
             'ment': 10,
             'ly': 7,
      

In [ ]:
accum_steering_words = t.zeros_like(accum_steering_sentences)

for (english, spanish) in words:
    cache_eng = gpt2_small.run_with_cache(english)[1]
    cache_spa = gpt2_small.run_with_cache(spanish)[1]

    for idx, key in enumerate(keys):
        vec_eng = cache_eng[key].squeeze()[-1]
        vec_spa = cache_spa[key].squeeze()[-1]
        steering = vec_spa - vec_eng
        accum_steering_words[idx] += steering

In [ ]:
steering_vecs_words = accum_steering_words / 50

def words_hook(res_vector, hook, steering):
    return res_vector + steering

for idx, key in enumerate(keys):
    partial_hook = functools.partial(words_hook, steering=steering_vecs_words[idx])
    logits = gpt2_small.run_with_hooks(
        'The horse drank some',
        return_type = 'logits',
        fwd_hooks = [(key, partial_hook)],
    )
    out = logits.squeeze().argmax(-1)[-1]
    print(gpt2_small.to_string(out))


### CAA [Paragraph]

### Scratch